# JSON Functions Testing

## Tasks to do:
* Load JSON file
* Dynamically update specified values within JSON file
* Export the updated JSON as new JSON file

In [12]:
import json

in_json = r"SACOG_ReduceVMT_template.json"

# load json file into python dict
with open(in_json, "r") as j_in:
    pyj = json.load(j_in)
    
print(json.dumps(pyj, indent=4))  # pretty print/format of json

{
    "charts": {
        "Jobs and Dwelling": {
            "title": "Total Jobs and Dwelling Units within 0.5mi of project",
            "features": [
                {
                    "attributes": {
                        "year": "2016",
                        "jobs": 1612,
                        "dwellingUnits": 3147
                    }
                },
                {
                    "attributes": {
                        "year": "2040",
                        "jobs": 2303,
                        "dwellingUnits": 4516
                    }
                }
            ]
        },
        "Land Use Diversity": {
            "title": "Land Use Diversity within 1mi of Project",
            "features": [
                {
                    "attributes": {
                        "type": "Project",
                        "diversity 2016": 0.51,
                        "diversity 2040": 0.56
                    }
                },
                {
           

In [14]:
    # "charts": {
    #     "Jobs and Dwelling": {
    #         "title": "Total Jobs and Dwelling Units within 0.5mi of project",
    #         "features": [
    #             {
    #                 "attributes": {
    #                     "year": "2016",
    #                     "jobs": 1612,
    #                     "dwellingUnits": 3147


# example updating values in JSON file
new_du_val = 999
pyj["charts"]["Jobs and Dwelling"]["features"][0]["attributes"]["dwellingUnits"] = new_du_val

print(json.dumps(pyj, indent=4))

{
    "charts": {
        "Jobs and Dwelling": {
            "title": "Total Jobs and Dwelling Units within 0.5mi of project",
            "features": [
                {
                    "attributes": {
                        "year": "2016",
                        "jobs": 1612,
                        "dwellingUnits": 999
                    }
                },
                {
                    "attributes": {
                        "year": "2040",
                        "jobs": 2303,
                        "dwellingUnits": 4516
                    }
                }
            ]
        },
        "Land Use Diversity": {
            "title": "Land Use Diversity within 1mi of Project",
            "features": [
                {
                    "attributes": {
                        "type": "Project",
                        "diversity 2016": 0.51,
                        "diversity 2040": 0.56
                    }
                },
                {
            

In [18]:
# output as new JSON file

out_json = json.dumps(pyj, indent=4) # indent spec needed so it doesn't come out as single line and is more readable

output_json = "test_json_out.json"

with open(output_json, 'w') as j_out:
    j_out.write(out_json)